In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

<div class="alert alert-block alert-info">
<b>To do</b>:
    
 - Sales de Mai 2023 indisponibles donc les supputer
 - Site à splitter pour récupérer plusieurs infos
 - Product Life cycel status
</div>

In [2]:
X = pd.read_csv('datasets_hi4/train-data.csv', sep=';')
X.drop_duplicates(inplace=True)

In [3]:
X['Reference proxy'] = X['Reference proxy'].apply(lambda x: x.replace('reference-', ''))
X['Product  Line proxy'] = X['Product  Line proxy'].apply(lambda x: x.replace('Product Line-', ''))
X['Division proxy'] = X['Division proxy'].apply(lambda x: x.replace('Division-', ''))
X['Customer Persona proxy'] = X['Customer Persona proxy'].apply(lambda x: x.replace('Customer Segmentation-', ''))
X['Strategic Product Family proxy'] = X['Strategic Product Family proxy'].apply(lambda x: x.replace('Strategic Product Family-', ''))

In [4]:
X[['Quarter', 'Year']] = X.Date.str.split(expand=True)
X['Quarter'] = X['Quarter'].map({'jan-apr': 1, 'may-jul': 2, 'may-aug': 2, 'sep-dec': 3})
X.drop('Date', axis=1, inplace=True)

In [5]:
df_melted = pd.melt(X[['Month 1', 'Month 2', 'Month 3', 'Month 4']], var_name='Month', value_name='Sales', ignore_index=False)
df_melted['Month'] = df_melted['Month'].str.replace('Month ', '').astype(int)
X = X.merge(df_melted, left_index=True, right_index=True)
X['Year'] = X['Year'].astype(int)
X['Month'] = X['Month'] + 4*(X['Quarter']-1)
X['Month_cumulated'] = (X['Month']-8) + (X['Year']-2020)*12
X['Month_cumulated'] = X['Month_cumulated'].map({33: 36, 34: 33, 35: 34, 36: 35}).fillna(X['Month_cumulated']).astype(int)
X.drop(['Month 1', 'Month 2', 'Month 3', 'Month 4', 'Quarter'], axis=1, inplace=True)
X = X[X['Month_cumulated']!=36]
X['Sales'] = X['Sales'].apply(lambda x: int(str(x).replace(' ', '')))
X.reset_index(drop=True, inplace=True)

In [6]:
X.columns

Index(['index', 'id_product', 'Region', 'Country', 'Site', 'Operations',
       'Zone', 'Cluster', 'Reference proxy', 'Product  Line proxy',
       'Division proxy', 'Customer Persona proxy',
       'Strategic Product Family proxy', 'Product Life cycel status', 'Year',
       'Month', 'Sales', 'Month_cumulated'],
      dtype='object')

In [7]:
X.head(3)

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Year,Month,Sales,Month_cumulated
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,13523,4,3,3,12,NaN,2021,5,0,9
1,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,13523,4,3,3,12,NaN,2021,6,0,10
2,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,13523,4,3,3,12,NaN,2021,7,0,11


In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6856015 entries, 0 to 6856014
Data columns (total 18 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   index                           int64 
 1   id_product                      int64 
 2   Region                          object
 3   Country                         object
 4   Site                            object
 5   Operations                      object
 6   Zone                            object
 7   Cluster                         object
 8   Reference proxy                 object
 9   Product  Line proxy             object
 10  Division proxy                  object
 11  Customer Persona proxy          object
 12  Strategic Product Family proxy  object
 13  Product Life cycel status       object
 14  Year                            int64 
 15  Month                           int64 
 16  Sales                           int64 
 17  Month_cumulated                 int64 
dtypes: